# Random Survival Forest, weighted

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import pickle

In [2]:
pd.set_option('display.max_columns', None,'display.max_rows',20)

In [3]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from lifelines.utils import datetimes_to_durations, concordance_index

In [4]:
# importing data
X_tr = pd.read_csv("D:/compiled_data/X_tr.csv").set_index("csid")

y = pd.read_csv("D:/compiled_data/y.csv").set_index("csid")

In [5]:
X_tr

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,pgf_x100000,il6_x100000,adgrg1_x100000,mcp_1_x100000,crtam_x100000,cxcl11_x100000,mcp_4_x100000,trail_x100000,fgf2_x100000,cxcl9_x100000,cd8a_x100000,caix_x100000,ifn_beta_x100000,ada_x100000,cd4_x100000,nos3_x100000,il2_x100000,gal_9_x100000,vegfr_2_x100000,cd40_x100000,il18_x100000,gzmh_x100000,vegfc_x100000,lap_tgf_beta_1_x100000,cxcl1_x100000,tnfsf14_x100000,il33_x100000,tweak_x100000,pdgf_subunit_b_x100000,pdcd1_x100000,faslg_x100000,cd28_x100000,ccl19_x100000,mcp_2_x100000,ccl4_x100000,il_35_x100000,gal_1_x100000,pd_l1_x100000,cd27_x100000,cxcl5_x100000,il5_x100000,hgf_x100000,gzma_x100000,ho_1_x100000,cx3cl1_x100000,cxcl10_x100000,cd70_x100000,il10_x100000,tnfrsf12a_x100000,ccl23_x100000,cd5_x100000,ccl3_x100000,mmp7_x100000,arg1_x100000,ncr1_x100000,dcn_x100000,tnfrsf21_x100000,tnfrsf4_x100000,mic_a_b_x100000,ccl17_x100000,angpt2_x100000,ptn_x100000,cxcl12_x100000,ifn_gamma_x100000,lamp3_x100000,casp_8_x100000,icoslg_x100000,mmp12_x100000,cxcl13_x100000,pd_l2_x100000,vegfa_x100000,il4_x100000,il_21_x100000,il12rb1_x100000,il13_x100000,ccl20_x100000,tnf_x100000,klrd1_x100000,gzmb_x100000,cd83_x100000,il12_x100000,csf_1_x100000,age_at_study_date_x100,household_size,tea_years_ago,alc_years_ago,smoking_today_amount,smoking_last_hour_amount,diabetes_diag_age,chd_diag_age,stroke_or_tia_diag_age,hypertension_diag_age,tb_diag_age,emph_bronc_diag_age,asthma_diag_age,cirrhosis_hep_diag_age,peptic_ulcer_diag_age,gall_diag_age,kidney_dis_diag_age,fracture_diag_age,rheum_arthritis_diag_age,psych_disorder_diag_age,neurasthenia_diag_age,head_injury_diag_age,cancer_diag_age,blood_transfusions,paid_blood_donations,children,siblings,mother_current_age,mother_age_at_death,father_current_age,father_age_at_death,siblings_stroke,siblings_diabetes,siblings_cancer,met,met_hours,met_work,met_work_hours,age_25_weight_jin,standing_height_mm,sitting_height_mm,waist_mm,hip_mm,waist_hip_ratio,weight_kg_x10,bmi_calc,impedance_ohms,fat_percent_x10,weight_etc_staff_code,sbp_mean,dbp_mean,heart_rate_mean,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10,qc_warning,precipitation,region_is_urban,is_female,has_health_cover,has_own_home,has_private_toilet,has_phone,has_motor_vehicle,had_recent_holiday,tea_drank_in_past,alc_today,alc_drank_in_past,smoking_today,smoking_100_plus_cigs,has_diabetes,has_copd,diabetes_diag,diabetes_now,chd_diag,chd_now,stroke_or_tia_diag,hypertension_diag,stroke_or_tia_now,hypertension_now,taking_aspirin,taking_ace_i,taking_beta_blocker,taking_diuretics,taking_statins,taking_ca_antagonist,taking_chlor_metaformin,taking_insulin,rheum_heart_dis_diag,tb_diag,tb_now,emph_bronc_diag,emph_bronc_now,asthma_diag,asthma_now,cirrhosis_hep_diag,cirrhosis_hep_now,peptic_ulcer_diag,peptic_ulcer_now,gall_diag,gall_now,kidney_dis_diag,kidney_dis_now,fracture_diag,fracture_now,rheum_arthritis_diag,rheum_arthritis_now,psych_disorder_diag,neurasthenia_diag,head_injury_diag,neurasthenia_now,cancer_diag,cancer_now,mother_stroke,mother_heart_attack,mother_diabetes,mother_mental_disorder,father_stroke,mother_cancer,father_diabetes,father_heart_attack,father_mental_disorder,father_cancer,siblings_heart_attack,siblings_mental_disorder,children_diabetes,children_cancer,children_mental_disorder,tried_to_reduce_weight,used_blood_pressure_drugs,region_code_12,region_code_16,region_code_26,region_code_36,region_code_46,region_code_52,region_code_58,region_code_68,region_code_78,region_code_88,highest_education_0,highest_education_1,highest_education_2,highest_education_3,highest_education_4,highest_education_5,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,marital_status_0,marital_status_1,marital_status_2,marital_status_3,househo

In [6]:
y

,ep_CKB0020_combined_ep,ep_CKB0020_combined_datedeveloped,study_date,subcohort
csid,,,,
2022-00229-1799093,1,2009-07-23T00:00:00Z,2006-04-17T10:08:09Z,False
2022-00229-4744170,1,2011-05-13T00:00:00Z,2006-09-25T15:24:06Z,False
2022-00229-1082517,1,2010-07-14T00:00:00Z,2004-12-16T11:22:31Z,False
2022-00229-3397875,1,2013-06-29T00:00:00Z,2006-07-25T08:54:40Z,False
2022-00229-4372175,1,2013-06-03T00:00:00Z,2007-05-09T09:58:21Z,False
...,...,...,...,...
2022-00229-5565114,0,2018-12-31T00:00:00Z,2006-09-09T09:46:34Z,True
2022-00229-4902412,0,2016-01-02T00:00:00Z,2006-07-24T09:08:24Z,True
2022-00229-4823882,0,2007-09-29T00:00:00Z,2005-04-16T09:56:21Z,True


In [7]:
def date_time_conversion(date):
    return(datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ"))

In [8]:
# Coverting the times into datetime format
y_tr = y.assign(
    event = y.ep_CKB0020_combined_ep,
    start_time = y.study_date.map(date_time_conversion),
    event_time = y.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y.columns)
y_tr["duration"] = datetimes_to_durations(y_tr["start_time"],y_tr["event_time"])[0]
y_tr["subcohort"] = y["subcohort"]
y_tr = y_tr.drop(columns = ["start_time","event_time"])
y_tr

,event,duration,subcohort
csid,,,
2022-00229-1799093,1,1192.0,False
2022-00229-4744170,1,1690.0,False
2022-00229-1082517,1,2035.0,False
2022-00229-3397875,1,2530.0,False
2022-00229-4372175,1,2216.0,False
...,...,...,...
2022-00229-5565114,0,4495.0,True
2022-00229-4902412,0,3448.0,True
2022-00229-4823882,0,895.0,True


## Fitting Model

First we need to find an appropriate resampling scheme.

In [10]:
from imblearn.over_sampling import RandomOverSampler

In [11]:
from weighted_concordance import *

In [12]:
ros = RandomOverSampler(sampling_strategy = {True: 1400 - sum(y_tr["subcohort"]), False: 512724 - (1400 - sum(y_tr["subcohort"])) })

In [13]:
case_subcohort = X_tr.join(y_tr).drop_duplicates()

In [14]:
covariates_resampled, event_resampled = ros.fit_resample(case_subcohort.drop(columns = ["event","subcohort"]), y_tr["event"])

C:\Users\thali\anaconda3\lib\site-packages\imblearn\utils\_validation.py:320: UserWarning: After over-sampling, the number of samples (806) in class True will be larger than the number of samples in the majority class (class #1 -> 595)
  warnings.warn(
C:\Users\thali\anaconda3\lib\site-packages\imblearn\utils\_validation.py:320: UserWarning: After over-sampling, the number of samples (511918) in class False will be larger than the number of samples in the majority class (class #1 -> 595)
  warnings.warn(


MemoryError: Unable to allocate 1.50 GiB for an array with shape (512724, 393) and data type float64

In [16]:
covariates_resampled                                                                                                                                                                                                                                                                  

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,pgf_x100000,il6_x100000,adgrg1_x100000,mcp_1_x100000,crtam_x100000,cxcl11_x100000,mcp_4_x100000,trail_x100000,fgf2_x100000,cxcl9_x100000,cd8a_x100000,caix_x100000,ifn_beta_x100000,ada_x100000,cd4_x100000,nos3_x100000,il2_x100000,gal_9_x100000,vegfr_2_x100000,cd40_x100000,il18_x100000,gzmh_x100000,vegfc_x100000,lap_tgf_beta_1_x100000,cxcl1_x100000,tnfsf14_x100000,il33_x100000,tweak_x100000,pdgf_subunit_b_x100000,pdcd1_x100000,faslg_x100000,cd28_x100000,ccl19_x100000,mcp_2_x100000,ccl4_x100000,il_35_x100000,gal_1_x100000,pd_l1_x100000,cd27_x100000,cxcl5_x100000,il5_x100000,hgf_x100000,gzma_x100000,ho_1_x100000,cx3cl1_x100000,cxcl10_x100000,cd70_x100000,il10_x100000,tnfrsf12a_x100000,ccl23_x100000,cd5_x100000,ccl3_x100000,mmp7_x100000,arg1_x100000,ncr1_x100000,dcn_x100000,tnfrsf21_x100000,tnfrsf4_x100000,mic_a_b_x100000,ccl17_x100000,angpt2_x100000,ptn_x100000,cxcl12_x100000,ifn_gamma_x100000,lamp3_x100000,casp_8_x100000,icoslg_x100000,mmp12_x100000,cxcl13_x100000,pd_l2_x100000,vegfa_x100000,il4_x100000,il_21_x100000,il12rb1_x100000,il13_x100000,ccl20_x100000,tnf_x100000,klrd1_x100000,gzmb_x100000,cd83_x100000,il12_x100000,csf_1_x100000,age_at_study_date_x100,household_size,tea_years_ago,alc_years_ago,smoking_today_amount,smoking_last_hour_amount,diabetes_diag_age,chd_diag_age,stroke_or_tia_diag_age,hypertension_diag_age,tb_diag_age,emph_bronc_diag_age,asthma_diag_age,cirrhosis_hep_diag_age,peptic_ulcer_diag_age,gall_diag_age,kidney_dis_diag_age,fracture_diag_age,rheum_arthritis_diag_age,psych_disorder_diag_age,neurasthenia_diag_age,head_injury_diag_age,cancer_diag_age,blood_transfusions,paid_blood_donations,children,siblings,mother_current_age,mother_age_at_death,father_current_age,father_age_at_death,siblings_stroke,siblings_diabetes,siblings_cancer,met,met_hours,met_work,met_work_hours,age_25_weight_jin,standing_height_mm,sitting_height_mm,waist_mm,hip_mm,waist_hip_ratio,weight_kg_x10,bmi_calc,impedance_ohms,fat_percent_x10,weight_etc_staff_code,sbp_mean,dbp_mean,heart_rate_mean,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10,qc_warning,precipitation,region_is_urban,is_female,has_health_cover,has_own_home,has_private_toilet,has_phone,has_motor_vehicle,had_recent_holiday,tea_drank_in_past,alc_today,alc_drank_in_past,smoking_today,smoking_100_plus_cigs,has_diabetes,has_copd,diabetes_diag,diabetes_now,chd_diag,chd_now,stroke_or_tia_diag,hypertension_diag,stroke_or_tia_now,hypertension_now,taking_aspirin,taking_ace_i,taking_beta_blocker,taking_diuretics,taking_statins,taking_ca_antagonist,taking_chlor_metaformin,taking_insulin,rheum_heart_dis_diag,tb_diag,tb_now,emph_bronc_diag,emph_bronc_now,asthma_diag,asthma_now,cirrhosis_hep_diag,cirrhosis_hep_now,peptic_ulcer_diag,peptic_ulcer_now,gall_diag,gall_now,kidney_dis_diag,kidney_dis_now,fracture_diag,fracture_now,rheum_arthritis_diag,rheum_arthritis_now,psych_disorder_diag,neurasthenia_diag,head_injury_diag,neurasthenia_now,cancer_diag,cancer_now,mother_stroke,mother_heart_attack,mother_diabetes,mother_mental_disorder,father_stroke,mother_cancer,father_diabetes,father_heart_attack,father_mental_disorder,father_cancer,siblings_heart_attack,siblings_mental_disorder,children_diabetes,children_cancer,children_mental_disorder,tried_to_reduce_weight,used_blood_pressure_drugs,region_code_12,region_code_16,region_code_26,region_code_36,region_code_46,region_code_52,region_code_58,region_code_68,region_code_78,region_code_88,highest_education_0,highest_education_1,highest_education_2,highest_education_3,highest_education_4,highest_education_5,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,marital_status_0,marital_status_1,marital_status_2,marital_status_3,househo

If we fit on the entire data set, computer memory runs out, therefore, we restrict the size of the subsamples to the size of the original dataset.

In [17]:
rsf = RandomSurvivalForest(bootstrap = True, max_samples = 1400)

In [18]:
y_tr_ros = Surv.from_arrays(event = event_resampled, time = covariates_resampled["duration"])

In [19]:
X_tr_ros = covariates_resampled.drop(columns = "duration")

In [20]:
# rsf.fit(X_tr_ros,y_tr_ros)

This fits fine.

In [21]:
from sklearn.model_selection import KFold

In [48]:
rf_n_estimator_scores = []

In [50]:
for n_estimators in [100,500,600]:
    
    kf = KFold(n_splits=5, shuffle = True)
    kf.get_n_splits(X_tr_ros)
    
    scores = []
    
    for train_index, test_index in kf.split(X_tr_ros):
        X_tr_train, X_tr_test = X_tr_ros.iloc[train_index], X_tr_ros.iloc[test_index]
        y_tr_train, y_tr_test = y_tr_ros[train_index], y_tr_ros[test_index]

        rsf = RandomSurvivalForest(bootstrap = True, max_samples = 1400)
        rsf.fit(X_tr_train,y_tr_train)
        
        scores.append(rsf.score(X_tr_test,y_tr_test))
        
        del rsf
        
    rf_n_estimator_scores.append(np.mean(scores))

In [51]:
rf_n_estimator_scores

[0.7232901220869163, 0.7149858652011061, 0.6975026274191188]

In [122]:
# Saving Model
pickle.dump(cph, open('unweighted_cox_pca.pkl', 'wb'))